# CN_Wheat: Nonlinear and memory-bound tasks

In [1]:
import pandas as pd
import numpy as np
import sys, os

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../'))  # for importing local packages from src

%reload_ext autoreload
%autoreload 2 

## Datasets

In [2]:
from src.model.rc_dataset import ExperimentDataset


DATASET_NEMA_H0 = 'datasets/dataset_NEMA_NEMA_H0.csv'
DATASET_NEMA_H3 = 'datasets/dataset_NEMA_NEMA_H3.csv'
DATASET_NEMA_H15 = 'datasets/dataset_NEMA_NEMA_H15.csv'

dataset_nema_h0 = ExperimentDataset(csv_path=DATASET_NEMA_H0)
dataset_nema_h3 = ExperimentDataset(csv_path=DATASET_NEMA_H3)
dataset_nema_h15 = ExperimentDataset(csv_path=DATASET_NEMA_H15)

## Model definition

Shared model parameters for all experiments:

- Ridge regresion model
  - Fitted bias term
  - Tuned regularization parameter
  - LeaveOneGroupOut cross-validation to tune param
- Datasets
  - NEMA_H0, H3 and H15 combined as one dataset
  - Grouped by calendar day of the model inputs
- Model preprocessing
  - ~~4 days of each run are discarded as warmup days~~ **Transforms must now be applied as a pipeline with the benchmark-specific transforms!**

In [3]:
DATASET_ID = 'NEMA_combined'

In [4]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneGroupOut
from src.learning.scorers import nmse_scorer
from pipeline_cnwheat import GroupGenerator, TimeGenerator
from pipeline_base import TrainTestSplitter, DirectTransform, WarmupTransform


readout_model = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])

model_param_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]

datasets = [
  dataset_nema_h0, 
  dataset_nema_h3,
  dataset_nema_h15
]

warmup_days = 4

shared_pipeline_params = {
  # Data generation
  'datasets': datasets,
  'groups': GroupGenerator(day_length=24),
  'time': TimeGenerator(day_length=24),
  
  # Model training and validation
  'readout_model': readout_model,
  'model_param_grid': model_param_grid,
  'model_scorer': nmse_scorer,
  'folds': LeaveOneGroupOut(),
  'train_test_split': TrainTestSplitter(block_size=4, test_ratio=0.5),
}

## Benchmark pipelines

### Pipeline definitions

#### Imports

In [5]:
from pipeline_base import (
  RCPipeline, 
  Rescale, 
  DaylightMask
)

from pipeline_cnwheat import (
  TargetGenerator, 
  SingleReservoirGenerator,
  MultiReservoirGenerator,
  TargetReservoirGenerator,
  GroupRescale
)

from model_config_cnwheat import (
  baseline_reservoirs,
  heterogeneous_reservoirs, 
  targets, 
  state_variables
)

#### Delay line benchmark

In [6]:
from pipeline_base import DelayLineTransform, CustomWarmupTransform

def delay_line_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, delay_steps: [int]):
  for d in delay_steps:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'delay_line',
        'delay': d,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        # NOTE: remove warmup before transform, otherwise the 
        # warmup steps are used as delayed prediction targets.
        WarmupTransform(warmup_days=4, day_length=24),
        DelayLineTransform(delay_steps=d),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline

#### Polynomial benchmark

In [7]:
from pipeline_base import PolynomialTargetTransform

def polynomial_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, exponents: [int]):
  for e in exponents:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'polynomial',
        'exponent': e,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        WarmupTransform(warmup_days=4, day_length=24),
        PolynomialTargetTransform(poly_coefs=[0] * e + [1]),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline

#### NARMA benchmark

In [8]:
from pipeline_base import NarmaTargetTransform

def narma_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, n_values: [int]):
  for n in n_values:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'NARMA',
        'narma_n': n,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        # NOTE: remove warmup before transform, otherwise the 
        # warmup steps are used as input to the NARMA system.
        WarmupTransform(warmup_days=4, day_length=24),  
        NarmaTargetTransform(n=n, scale=1),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline

#### Near future prediction

In [9]:
def future_pred_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, lookahead_steps: [int]):
  for l in lookahead_steps:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'near_future_pred',
        'lookahead': l,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        # NOTE: remove warmup before transform, otherwise the 
        # warmup steps are used as delayed prediction targets.
        WarmupTransform(warmup_days=4, day_length=24),
        DelayLineTransform(delay_steps=-l),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline

In [10]:
test_target = TargetGenerator(target=targets[0])
test_reservoir = SingleReservoirGenerator(state_var=state_variables[0])
rescale = Rescale(per_feature=False)

lookahead_steps = [0, 1, 2]

test_narma_pipelines = list(future_pred_benchmark(test_target, 'target', test_reservoir, 'res', rescale, lookahead_steps=lookahead_steps))

### Generate pipelines

In [11]:
from itertools import chain

benchmarks = [
  {
    'benchmark': delay_line_benchmark,
    'kwargs': {
      'delay_steps': np.arange(0, 13)  # 0-12 hours
    }
  },
  {
    'benchmark': future_pred_benchmark,
    'kwargs': {
      'lookahead_steps': np.arange(0, 7)  # 0-6 hours
    }
  },
  {
    'benchmark': polynomial_benchmark,
    'kwargs': {
      'exponents': np.arange(1, 10)  # exponents 1-9
    }
  },
  {
    'benchmark': narma_benchmark,
    'kwargs': {
      'n_values': [2, 5, 10, 20, 50, 100]
    }
  },
]


def generate_benchmark_pipelines(target_gen, target_id, reservoir_gen, reservoir_id, rescale):
  pipeline_generators = []
  for benchmark in benchmarks:
    gen = benchmark['benchmark'](target_gen, target_id, reservoir_gen, reservoir_id, rescale, **benchmark['kwargs'])
    pipeline_generators.append(gen)
  return chain(*pipeline_generators)


In [12]:
target_state_pairs = [(target, state_var) for target in targets for state_var in state_variables]
target_env_pairs = [(name, target, env_targets) for target in targets for (name, env_targets) in baseline_reservoirs]
target_het_pairs = [(name, target, state_vars) for target in targets for (name, state_vars) in heterogeneous_reservoirs]


all_pipelines = []

for target, state_var in target_state_pairs:
  target_gen = TargetGenerator(target=target)
  res_gen = SingleReservoirGenerator(state_var=state_var)
  pipelines = generate_benchmark_pipelines(
    target_gen, target, res_gen, state_var, 
    Rescale(per_feature=False)
  )
  all_pipelines += pipelines

for name, target, state_vars in target_het_pairs:
  target_gen = TargetGenerator(target=target)
  res_gen = MultiReservoirGenerator(state_vars=state_vars)
  pipelines = generate_benchmark_pipelines(
    target_gen, target, res_gen, name, 
    GroupRescale(datasets=datasets, state_vars=state_vars)
  )
  all_pipelines += pipelines

for name, target, env_targets in target_env_pairs:
  target_gen = TargetGenerator(target=target)
  res_gen = TargetReservoirGenerator(targets=env_targets)
  pipelines = generate_benchmark_pipelines(
    target_gen, target, res_gen, name, 
    Rescale(per_feature=True)
  )
  all_pipelines += pipelines


print(len(all_pipelines))

4725


### Execution

In [13]:
from tqdm import tqdm
from itertools import chain
from pipeline_base import execute_pipeline


RESULTS_FILE = 'results_cnwheat_benchmarks.csv'

total_loops = len(all_pipelines)
results = []


with tqdm(total=total_loops) as pbar:
  for pipeline in all_pipelines:

    try:
      result = execute_pipeline(pipeline)
      results.append(result)
      pbar.update(1)
    except Exception as e:
      print('An exception occured executing the pipeline with the following metadata:')
      print(f'{pipeline.metadata}')
      raise e


results_df = pd.DataFrame.from_dict(results)
results_df.to_csv(RESULTS_FILE)
print(f'Saved scores to {RESULTS_FILE}')

100%|██████████| 4725/4725 [30:51<00:00,  2.55it/s]

Saved scores to results_cnwheat_benchmarks.csv
